# NB3: Notebook created for processing judgment text

**Output:** Clean dataset with the following variables. 

- [ ] `decision_date`: application date. 
- [ ] `referral_date `: date when the reference for preliminary referenc was sent. 
- [ ] `referring_court_name`: ???
- [ ] `full_text`: 
- [ ] `par_location`:
- [ ] 

**Output**: Create script that scrapes updates

## Environment and Settings for Python

In [2]:
# REQUIRED  Libraries
import os 
import sys
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

# ADD project paths for for easier access across whole project
#paths_script_path = "C:\\Users\\mauricmm\\iCloudDrive\\cloudgit\\uio24emc\\"
parent_dir = os.path.dirname(os.getcwd(),) +"\\"
sys.path.append(parent_dir)
import paths

# Access paths globally
paths.figures_dir

'c:\\gitprojects\\uio24emc\\figures\\'

## Start Processing Judgements Text

In [3]:
#READ IUROPA Text Corpus Data
iuropa_corpus_path = "C:\\gitprojects\\uio24emc\\raw\\iuropa_text.gz.parquet"
iuropa_corpus = pq.read_table(iuropa_corpus_path)
iuropa_corpus_df = iuropa_corpus.to_pandas()
schema = pq.read_schema(iuropa_corpus_path)

In [4]:
# [ ] CREATE columns selector 
# [ ] CREATE rows boolean mask
# [ ] CHECK if the text from the sections is good  to process. 
# [ ] OUTPUT target dataset to to data as  "judgements" 
iuropa_corpus_df.columns


# Columns selector Selector
cols_selector =  ['document_id', 'paragraph_id', 'language', 'ecli', 'court',
       'date', 'year', 'text', 'line_id', 'line_id_prop', 'section',
       'paragraph_type', 'paragraph_number', 'nchar', 'html_class',
       'html_attr']

# BOOLEAN Conditions for mask



In [13]:
iuropa_corpus_df.head()
iuropa_corpus_df

# Search for rows where the specified column contains the search text
# This example performs a case-insensitive search
limited_df = iuropa_corpus_df.head(1000)
matching_rows = limited_df[limited_df['text'].str.contains( "admissibility" , case=False, na=False)]

### NOTES

In [ ]:
# USE Query for complex but not to elaborated queries
filtered_df = df.query('description.str.contains("data") & id > 1', engine='python')

# USE BOOLEAN masks for more complex conditions
condition1 = df['description'].str.contains('data', case=False, na=False)
condition2 = df['age'] > 30
condition3 = df['name'].isin(['Alice', 'David'])
# Combine the conditions using logical operators
complex_mask = condition1 & condition2 | condition3

# Apply the mask to filter the DataFrame
filtered_df = df[complex_mask]